In [1]:
import os
import sys
#spark_home = os.environ['SPARK_HOME'] = 'C:\\spark-1.5.1-bin-hadoop2.4'
spark_home = os.environ['SPARK_HOME'] = 'C:\\spark-1.6.1-bin-hadoop2.6'

if not spark_home:
    raise ValueError('SPARK_HOME enviroment variable is not set')
sys.path.insert(0,os.path.join(spark_home,'python'))
#sys.path.insert(0,os.path.join(spark_home,'python', 'lib', 'py4j-0.8.2.1-src.zip'))
sys.path.insert(0,os.path.join(spark_home,'python', 'lib', 'py4j-0.9-src.zip'))
execfile(os.path.join(spark_home,'python', 'pyspark', 'shell.py'))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 1.6.1
      /_/

Using Python version 2.7.11 (default, Jan 29 2016 14:26:21)
SparkContext available as sc, HiveContext available as sqlContext.


# HW11.2 Gradient descent

In the context of logistic regression describe and define three flavors of penalized loss functions.  
Are these all supported in Spark MLLib (include online references to support your answers)?

Descibe probabilitic interpretations of the L1 and L2 priors for penalized logistic regression
(HINT: see synchronous slides for week 11 for details)

In the context of logistic regression, the three flavors of penalized terms are:
- L1 Reg, which penalizes for sum of absolute weights:
![alt text](http://people.ischool.berkeley.edu/~kuanlin/l1_reg.PNG "L1 Reg")
- L2 Reg, penalizes sum of squared weights:
![alt text](http://people.ischool.berkeley.edu/~kuanlin/l2_reg.PNG "L2 Reg")
- Elastic Net, penalizes a linear combination of L1 and L2 norms:
![alt text](http://people.ischool.berkeley.edu/~kuanlin/elastic_net.PNG "Elastic Net Reg")

All of the above three regularization methods are supported by spark.mllib:<br/>
http://spark.apache.org/docs/latest/mllib-linear-methods.html#regularizers

#### Probablisitic interpretation of L1 and L2 priors:

L1 regularization can be interpreted as using Laplace distribution as the prior distribution for the model weights, where as L2 regularization can be interpreted as using gaussian distribution as the prior distribution for the model weights.
![alt text](http://people.ischool.berkeley.edu/~kuanlin/gaussian_vs_laplace.PNG "Distribution Overlay")

The Laplace distribution has more density closer to mean (usually zero in most settings) in comparison to the gaussian distribution, and therefore L1 regularization will tend to push the model weights toward zero.

In [2]:
import numpy as np
np.random.seed(0)
def generateData2(n):
    """ 
    non-linearly seperable data
    """
    xb = np.random.normal(0,0.5,n)-0.5
    yb = np.random.normal(0,0.5,n)+0.5
    xr = np.random.normal(0,0.5,n)+0.5
    yr = np.random.normal(0,0.5,n)-0.5
    inputs = []
    for i in range(len(xb)):
        inputs.append([xb[i],yb[i],1])
        inputs.append([xr[i],yr[i],-1])
    return inputs

data_lin_inseperable_train = generateData2(50) # train data
data_lin_inseperable_test = generateData2(50) # test data

# 11.5  [OPTIONAL] Distributed Perceptron algorithm.

Using the following papers as background:
http://static.googleusercontent.com/external_content/untrusted_dlcp/research.google.com/en//pubs/archive/36266.pdf

https://www.dropbox.com/s/a5pdcp0r8ptudgj/gesmundo-tomeh-eacl-2012.pdf?dl=0

http://www.slideshare.net/matsubaray/distributed-perceptron 

Implement each of the following flavors of perceptron learning algorithm:

1. Serial (All Data): This is the classifier returned if trained serially on all the 
available data.  On a single computer for example (Mistake driven)
2. Serial (Sub Sampling): Shard the data, select one shard randomly and train serially. 
3. Parallel (Parameter Mix): Learn a perceptron locally on each shard: 
Once learning is complete combine each learnt percepton using a uniform weighting
4. Parallel (Iterative Parameter Mix) as described in the above papers.

#### Perceptron Gradient Descent

In [64]:
import numpy as np

def perceptronSubGradientCalc(data, w, regParam, feature_size):
    #gradient = data.map(lambda p: -p.y*np.array(p.x) if (p.y*np.dot(w.value,p.x))<0 else np.zeros(feature_size)).reduce(lambda a, b: a+b)
    incorrect = data.filter(lambda p: np.dot(w.value,p.x)*p.y < 0)
    if incorrect.isEmpty():
        gradient = np.zeros(feature_size)
    else:
        gradient = incorrect.map(lambda p: -p.y*np.array(p.x)).reduce(lambda a,b: a+b)
    wreg = np.array(w.value)  # L2 reg
    wreg[-1] = 0 # don't count the bias term in reg
    return gradient/data.count() + regParam*wreg

def perceptronAccuracy(data, w):
    correct_count = data.map(lambda p: 1 if np.dot(w.value, p.x)*p.y > 0 else 0).reduce(lambda a,b: a+b)
    return 1.0*correct_count/data.count()

In [7]:
from collections import namedtuple
import numpy as np
Point = namedtuple('Point', 'x y')

def readPoint(data):
    label = data[2]
    x = [data[0], data[1], 1.0]  #add bias term
    return Point(x, label)

train_data = sc.parallelize(data_lin_inseperable_train).map(readPoint).cache()
test_data = sc.parallelize(data_lin_inseperable_test).map(readPoint).cache()

#train_data.count()

#### Train serially on all data

In [65]:
def perceptronGDSerialAll(data, wInitial=None, nShards=1, nIter=10, stopCriteria=0.001, 
                 learningRate=0.05, regParam=0.01):
    feature_size = len(data.take(1)[0].x)
    if wInitial is None:
        w = np.random.normal(size=feature_size)
    else:
        w = wInitial
    #model_data = data.randomSplit([1.0/nShards]*nShards) # split data into shards
    for i in range(nIter):
        wb = sc.broadcast(w) # broadcast weight
        wdelta = learningRate*perceptronSubGradientCalc(data, wb, regParam, feature_size)
        if sum(abs(wdelta))<=stopCriteria*sum(abs(w)):
            break
        w = w - wdelta
    return w

w = perceptronGDSerialAll(train_data)
print "learned weight: %s" % str(w)
print "Accuracy: %s" % perceptronAccuracy(test_data, sc.broadcast(w))

learned weight: [-2.16751633  0.22811287 -0.02710354]
Accuracy: 0.88


### Train serially on random shards

In [80]:
def perceptronGDSerialShards(data, wInitial=None, nShards=1, nIter=10, stopCriteria=0.001, 
                 learningRate=0.05, regParam=0.01):
    feature_size = len(data.take(1)[0].x)
    if wInitial is None:
        w = np.random.normal(size=feature_size)
    else:
        w = wInitial
    for shared_data in data.randomSplit([1.0/nShards]*nShards): # split data into shards
        for i in range(nIter):
            wb = sc.broadcast(w) # broadcast weight
            wdelta = learningRate*perceptronSubGradientCalc(shared_data, wb, regParam, feature_size)
            if sum(abs(wdelta))<=stopCriteria*sum(abs(w)):
                break
            w = w - wdelta
    return w

w = perceptronGDSerialShards(train_data, nShards=4, nIter=50)
print "learned weight: %s" % str(w)
print "Accuracy: %s" % perceptronAccuracy(test_data, sc.broadcast(w))

learned weight: [-0.0394292   0.17467005 -0.00799986]
Accuracy: 0.9


#### Parameter Mixing Method (non-iterative)

In [78]:
def percetronTrainLocal(data, w, regParam, feature_size, learningRate, nIter):
    # train with local data instead of RDD
    weight = np.array(w.value)
    for i in range(nIter):
        gradient = sum(map(lambda p: -p.y*np.array(p.x) if (p.y*np.dot(weight,p.x))<0 else np.zeros(feature_size), data))
        wreg = weight*1  # L2 reg
        wreg[-1] = 0 # don't count the bias term in reg
        weight -= learningRate*(gradient/len(data) + regParam*wreg)
    return weight

def perceptronGDSerialParaMix(data, wInitial=None, nShards=1, nIter=10, learningRate=0.05, regParam=0.01):
    feature_size = len(data.take(1)[0].x)
    if wInitial is None:
        w = np.random.normal(size=feature_size)
    else:
        w = wInitial
    wb = sc.broadcast(w)
    model_data = sc.parallelize([d.collect() for d in data.randomSplit([1.0/nShards]*nShards)]) # split data into shards in RDD
    learned_w = model_data.map(lambda data: percetronTrainLocal(data, wb, regParam, feature_size, learningRate, nIter)).collect()
    return sum(learned_w)/len(learned_w) # take uniform average of the learned weights

w = perceptronGDSerialParaMix(train_data, nShards=4, nIter=50)
print "learned weight: %s" % str(w)
print "Accuracy: %s" % perceptronAccuracy(test_data, sc.broadcast(w))

learned weight: [ 0.82557121  2.61303986 -0.09620172]
Accuracy: 0.79


#### Iterative Param Mixing

In [77]:
def perceptronGDSerialIterParaMix(data, wInitial=None, nShards=1, nIter=10, learningRate=0.05, regParam=0.01):
    feature_size = len(data.take(1)[0].x)
    if wInitial is None:
        w = np.random.normal(size=feature_size)
    else:
        w = wInitial
    model_data = sc.parallelize([d.collect() for d in data.randomSplit([1.0/nShards]*nShards)]) # split data into shards in RDD
    for i in range(nIter):
        wb = sc.broadcast(w)
        # train only 1 epoch and then do param mixing
        learned_w = model_data.map(lambda data: percetronTrainLocal(data, wb, regParam, feature_size, learningRate, 1)).collect()
        w = sum(learned_w)/len(learned_w) # take uniform average of the learned weights
    return w

w = perceptronGDSerialIterParaMix(train_data, nShards=4, nIter=50)
print "learned weight: %s" % str(w)
print "Accuracy: %s" % perceptronAccuracy(test_data, sc.broadcast(w))

learned weight: [-1.16103408  0.03655426  0.44536896]
Accuracy: 0.74
